In [ ]:
import os
import yaml
import torch
from ultralytics import YOLO

# Check for GPU
if torch.cuda.is_available():
    print(f"✅ GPU Detected: {torch.cuda.get_device_name(0)}")
    device = 0
else:
    print("⚠️ No GPU detected. Evolution will be extremely slow on CPU.")
    device = 'cpu'

In [ ]:
# --- Dataset Path Configuration 
cwd = os.getcwd()
possible_paths = [
    os.path.join(cwd, 'oral-diseases-1'),
    os.path.join(cwd, 'Backend Development', 'Model B', 'oral-diseases-1'),
    os.path.join(cwd, 'Model B', 'oral-diseases-1')
]

dataset_dir = None
for path in possible_paths:
    if os.path.exists(os.path.join(path, 'data.yaml')):
        dataset_dir = path
        break

if dataset_dir is None:
    raise FileNotFoundError("Could not find 'oral-diseases-1' dataset folder.")

yaml_path = os.path.join(dataset_dir, 'data.yaml')
print(f"✅ Using dataset at: {yaml_path}")

# Fix paths in data.yaml to be absolute
with open(yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

data_config['train'] = os.path.join(dataset_dir, 'train', 'images')
data_config['val'] = os.path.join(dataset_dir, 'valid', 'images')
data_config['test'] = os.path.join(dataset_dir, 'test', 'images')

with open(yaml_path, 'w') as f:
    yaml.dump(data_config, f)
print("✅ data.yaml paths updated to absolute paths.")

In [ ]:
# --- Run Evolution ---
# We start with your pretrained model (best.pt) to find the best hyperparameters for fine-tuning it further.
# If you wanted to find the best parameters for training from scratch, you would use 'yolov8m.pt' here.
model_path = 'models/best.pt'
if not os.path.exists(model_path):
    print(f"⚠️ '{model_path}' not found. Falling back to 'yolov8m.pt'")
    model_path = 'yolov8m.pt'

model = YOLO(model_path)

# Determine workers based on OS (8 for Linux/HPC, 0 for Windows)
workers = 8 if os.name != 'nt' else 0

print(f"Starting hyperparameter evolution on {model_path}...")
print("This will run multiple short training sessions to find the optimal settings.")

# Tune hyperparameters
model.tune(
    data=yaml_path,
    epochs=30,          # Train for 30 epochs per iteration (short runs)
    iterations=50,      # Run 50 genetic evolution iterations
    optimizer='AdamW',  # Optimizer to use
    plots=False, 
    save=False,
    val=True,
    device=device,
    batch=-1,           # ✅ AutoBatch: Automatically finds the max batch size for your GPU
    workers=workers     # ✅ Max Workers: Speeds up data loading
)

print("Evolution complete. The best hyperparameters are saved in the 'runs/detect/tune' folder.")
print("You can use these parameters to run a final, long training session.")

In [ ]:
# --- Step 4: Final Training with Evolved Hyperparameters ---
# ⚠️ ONLY RUN THIS AFTER THE EVOLUTION STEP ABOVE FINISHES ⚠️

import glob

# 1. Find the best hyperparameters file
# The evolution saves results in runs/detect/tune/best_hyperparameters.yaml
# We search for it dynamically just in case
tune_dirs = glob.glob('runs/detect/tune*')
best_hyperparams_path = None

if tune_dirs:
    # Get the most recent tune folder
    latest_tune_dir = max(tune_dirs, key=os.path.getmtime)
    potential_path = os.path.join(latest_tune_dir, 'best_hyperparameters.yaml')
    if os.path.exists(potential_path):
        best_hyperparams_path = potential_path

if best_hyperparams_path:
    print(f"✅ Found best hyperparameters at: {best_hyperparams_path}")
    with open(best_hyperparams_path, 'r') as f:
        hyperparams = yaml.safe_load(f)
else:
    print("⚠️ No hyperparameter file found. Did the evolution finish?")
    print("Using default settings for now.")
    hyperparams = {}

# 2. Train the Final Model
# We train for longer (e.g., 100-150 epochs) to get the best possible accuracy
print("Starting Final Training with Optimized Hyperparameters...")

final_model = YOLO('models/best.pt')  # Load the best model from previous training

final_model.train(
    data=yaml_path,
    epochs=150,         # Train for longer to maximize accuracy
    patience=20,        # Stop if no improvement for 20 epochs
    batch=-1,           # Use AutoBatch again
    device=device,
    project='oral_cancer_screening',
    name='yolov8m_evolved_final',
    exist_ok=True,
    workers=8,          # Keep high workers for HPC
    **hyperparams       # <--- This applies the evolved settings!
)

print("✅ Final model training complete! Your best model is in 'oral_cancer_screening/yolov8m_evolved_final/weights/best.pt'")

## 4. Final Training (After Evolution)

Once the evolution process above finishes (which may take hours), it will save the best hyperparameters to a file.

**Run the cell below** to automatically load those "best settings" and train your final, high-performance model.